In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
pd.set_option('display.max_columns', None) # optional - only for this example

loading the file from excel
- sheet is different from the default one
- data has some not usefull records, which will be skipped

In [ ]:
df = pd.read_excel('Canada.xlsx', sheet_name=1, skiprows=20, skipfooter=2)
df.head()

summarize data

In [ ]:
df.info()

check missing entry

In [ ]:
df.isnull().sum()

check the names of numerical and categorical columns

In [ ]:
print('columns with numeric data')
print(df.select_dtypes(include='number').columns.tolist())
print('columns with non numerica data')
print(df.select_dtypes(exclude='number').columns.tolist())

describe numerical data

In [ ]:
df.describe() # by default numeric columns are considered

describe non-numerical data

In [ ]:
df.describe(exclude='number')

### data wrangling

In [ ]:
df.head(10)

drop columns that are not required

In [ ]:
cols_to_drop = ['Type', 'Coverage', 'AREA','DEV','REG']
df.drop(columns=cols_to_drop, inplace=True)
df.head()

renaming the columns to something meaningful

In [ ]:
rename_dict = {'OdName':'Country',
                'AreaName':'Continent',
                'RegName':'Region',
                'DevName':'Status'}
df.rename(columns=rename_dict, inplace=True)
df.head()

add country wise total immigration

In [ ]:
years = list(range(1980,2014))
df['Total'] = df[years].sum(axis=1)
df.head()

since each country is unique, i will use country as index

In [ ]:
df.set_index('Country', inplace=True)
df.head()